## 1.1 Download data

In [7]:
# Data of "Political Stability and Absence of Violence/Terrorism: Estimate" and "GDP growth (annual %)"
import requests
import pandas as pd

def fetch_world_bank_data(indicator_code):
    url = f"https://api.worldbank.org/v2/en/indicator/{indicator_code}?format=json&per_page=20000"
    response = requests.get(url)
    data = response.json()[1] 

## 1.2 Manipulate Data

## 1.3 Merge Data